# Locality Sensitive Hashing

Practical course material for the ASDM Class 09 (Text Mining) by Florian Leitner.

© 2016 Florian Leitner. All rights reserved.

In [115]:
from collections import defaultdict
# lower the number of digits shown for floating-point numbers:
%precision %e

'%e'

## Banker's rounding

Casting to an `int` chops off the decimals: 

In [2]:
int(1.01), int(1.5), int(1.99), int(2.5), int(3.5)

(1, 1, 1, 2, 3)

But if using `round(float)` the you get Banker's rounding (on even final digit it rounds down, on uneven digits up):

In [3]:
round(1.01), round(1.5), round(1.99), round(2.5), round(3.5)

(1, 2, 2, 2, 4)

Why should this bother you? For example:

In [4]:
[round(i/2) for i in range(1,5)]

[0, 1, 2, 2]

In [5]:
[i//2 for i in range(1,5)]

[0, 1, 1, 2]

In case you are surprised by this behaviour; It is the standard [IEEE 754](https://en.wikipedia.org/wiki/Floating_point#Rounding_modes) behaviour for floating points that should be followed by all programming languages; It's languages that *don't* exhibit this bevahiour that are "wrong".

## The `defaultdict`

Quick review of `deafultdict`:

In [48]:
demo = defaultdict(int)
demo['defined'] = demo['defined'] + 1 # NB: we did not define a value for 'defined', it magically initialized itself!
demo['missing'], demo['defined']

(0, 1)

All practicality aside, this can become a trap, too:

In [49]:
'missing' in demo

True

In [50]:
'really missing' in demo

False

## Jaccard-based word similarity

In [51]:
def shingle(s, k):
    """Generate k-length shingles of string s."""
    k = min(len(s), k)
    for i in range(len(s) - k + 1):
        yield s[i:i+k]

def hshingle(s, k):
    """Generate k-length shingles then hash."""
    for s in shingle(s, k):
        yield hash(s)

def jaccard(X, Y):
    """The Jaccard similarity between two sets."""
    x = set(X)
    y = set(Y)
    return float(len(x & y)) / len(x | y)

Create all character trigrams of a string:

In [52]:
list(shingle('string', 3))

['str', 'tri', 'rin', 'ing']

Calculate the integer hash of a string:

In [56]:
hash('str')

-7191288385591411446

In [55]:
list(hshingle('string', 3))

[-7191288385591411446,
 -675603213479884509,
 -7932968880153686062,
 -4052543708567640597]

Character unigram similarity:

In [57]:
jaccard('string', 'string'), jaccard('string', 'strang'), jaccard('string', 'other')

(1.000, 0.714, 0.222)

In [58]:
jaccard('word', 'wirt')

0.333

In [59]:
jaccard('alabama', 'malba') # not so good...

1.000

Therefore, it is better to use higher order; Bigram similarity:

In [60]:
jaccard(frozenset(shingle('alabama', 2)), frozenset(shingle('malba', 2)))

0.429

Or even character trigrams:

In [61]:
jaccard(frozenset(shingle('alabama', 3)), frozenset(shingle('malba', 3)))

0.000

In [62]:
jaccard(frozenset(shingle('word', 3)), frozenset(shingle('wirt', 3))) # not so good, either...

0.000

In [63]:
jaccard('karlos', 'carol')

0.571

In [64]:
jaccard(frozenset(shingle('karlos', 2)), frozenset(shingle('carol', 2)))

0.125

In the end, depending on the scenario, a problem-specific decision for using uni-, bi- or, maybe a trigram sets has to be made.

## Implementing minhash

We will look into an implementation of LSH by [Christian Jauvin](http://cjauvin.github.io/); see [github.com/go2starr/lshhdc](https://github.com/go2starr/lshhdc).

In [65]:
class MinHashSignature:
    """Hash signatures for sets/tuples using minhash."""

    def __init__(self, dim):
        """
        Define the dimension of the hash pool
        (number of hash functions).
        """
        self.dim = dim
        self.hashes = self.hash_functions()

    def hash_functions(self):
        """Return dim different hash functions."""
        def hash_factory(n):
            return lambda x: hash("salt" + str(n) + str(x) + "salt")
        
        return [ hash_factory(_) for _ in range(self.dim) ]

    def sign(self, item):
        """Return the minhash signatures for the `item`."""
        sig = [ float("inf") ] * self.dim
        
        for hash_ix, hash_fn in enumerate(self.hashes):
            # minhashing; requires item is iterable:
            sig[hash_ix] = min(hash_fn(i) for i in item)
        
        return sig

A MinHash signature that generates four different hash values ("dimensions") for the same string:

In [66]:
sig4 = MinHashSignature(4)
sig4.sign("example")

[-7288557722884770586,
 -6492668138025979145,
 -8490456313657407868,
 -5809476266454031930]

## Locality Sensitive Hashing

In [20]:
class LSH:
    """
    Locality sensitive hashing.
    
    Uses a banding approach to hash
    similar signatures to the same buckets.
    """
    
    def __init__(self, size, threshold):
        """
        LSH approximating a given similarity `threshold`
        with a given hash signature `size`.
        """
        self.size = size
        self.threshold = threshold
        self.bandwidth = self.get_bandwidth(size, threshold)

    @staticmethod
    def get_bandwidth(n, t):
        """
        Approximate the bandwidth (number of rows in each band)
        needed to get threshold.

        Threshold t = (1/b) ** (1/r)
        where
        b = # of bands
        r = # of rows per band
        n = b * r = size of signature
        """
        best = n # 1
        minerr = float("inf")
        
        for r in range(1, n + 1):
            try:
                b = 1. / (t ** r)
            except: # Divide by zero, your signature is huge
                return best
            
            err = abs(n - b * r)
            
            if err < minerr:
                best = r
                minerr = err
                
        return best

    def hash(self, sig):
        """Generate hash values for this signature."""
        for band in zip(*(iter(sig),) * self.bandwidth):
            yield hash("salt" + str(band) + "tlas")

    @property
    def exact_threshold(self):
        """The exact threshold defined by the chosen bandwith."""
        r = self.bandwidth
        b = self.size / r
        return (1. / b) ** (1. / r)

    def get_n_bands(self):
        """The number of bands."""
        return int(self.size / self.bandwidth)

### Helper class: UnionFind

This is an additional data structure to unite the equal bins in the disperesd bands into one virtual set. Please refer to the free Book by Rajaraman, "Mining of Massive Datasets", for more information why and how this works.

In [21]:
class UnionFind:
    """
    Union-find data structure.

    Each unionFind instance X maintains a family of disjoint sets of
    hashable objects, supporting the following two methods:

    - X[item] returns a name for the set containing the given item.
    Each set is named by an arbitrarily-chosen one of its members; as
    long as the set remains unchanged it will keep the same name. If
    the item is not yet part of a set in X, a new singleton set is
    created for it.

    - X.union(item1, item2, ...) merges the sets containing each item
    into a single larger set. If any item is not yet part of a set
    in X, it is added to X as one of the members of the merged set.
    
    Source: http://www.ics.uci.edu/~eppstein/PADS/UnionFind.py

    Union-find data structure. Based on Josiah Carlson's code,
    http://aspn.activestate.com/ASPN/Cookbook/Python/Recipe/215912
    with significant additional changes by D. Eppstein.
    """

    def __init__(self):
        """Create a new empty union-find structure."""
        self.weights = {}
        self.parents = {}

    def __getitem__(self, object):
        """Find and return the name of the set containing the object."""
        # check for previously unknown object
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
            
        return root

    def __iter__(self):
        """Iterate through all items ever found or unioned by this structure."""
        return iter(self.parents)

    def union(self, *objects):
        """Find the sets containing the objects and merge them all."""
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r],r) for r in roots])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest

    def sets(self):
        """Return a list of each disjoint set"""
        ret = defaultdict(list)
        for k, _ in self.parents.items():
            ret[self[k]].append(k)
        return list(ret.values())

Just to give you a quick and dirty understanding of what it does:

In [68]:
uf = UnionFind()
uf.union(0, 1)
print('sets', uf.sets())
print('name of containing set:', ''.join("\nitem %i belongs to set %i" % (i, uf[i]) for i in uf))

sets [[0, 1]]
name of containing set: 
item 0 belongs to set 1
item 1 belongs to set 1


Adding and uniting two new items to the set produces two independent sets:

In [69]:
uf.union(2, 3)
print('sets', uf.sets())
print('name of containing set:', ''.join("\nitem %i belongs to set %i" % (i, uf[i]) for i in uf))

sets [[0, 1], [2, 3]]
name of containing set: 
item 0 belongs to set 1
item 1 belongs to set 1
item 2 belongs to set 3
item 3 belongs to set 3


If we unite two items in two disjoint sets, the sets are merged:

In [70]:
uf.union(3, 0)
print('sets', uf.sets())
assert uf.sets() == [[0, 1, 2, 3]], uf.sets()
print('name of containing set:', ''.join("\nitem %i belongs to set %i" % (i, uf[i]) for i in uf))

sets [[0, 1, 2, 3]]
name of containing set: 
item 0 belongs to set 3
item 1 belongs to set 3
item 2 belongs to set 3
item 3 belongs to set 3


Looking up an item not yet in a set puts it in its own, new set:

In [71]:
uf[4]
print('sets', uf.sets())
print('name of containing set:', ''.join("\nitem %i belongs to set %i" % (i, uf[i]) for i in uf))

sets [[0, 1, 2, 3], [4]]
name of containing set: 
item 0 belongs to set 3
item 1 belongs to set 3
item 2 belongs to set 3
item 3 belongs to set 3
item 4 belongs to set 4


In [73]:
uf[3]
print('sets', uf.sets())
print('name of containing set:', ''.join("\nitem %i belongs to set %i" % (i, uf[i]) for i in uf))

sets [[0, 1, 2, 3], [4]]
name of containing set: 
item 0 belongs to set 3
item 1 belongs to set 3
item 2 belongs to set 3
item 3 belongs to set 3
item 4 belongs to set 4


### A banded LSH implementation

In [74]:
class Cluster:
    """
    Cluster items with a Jaccard similarity above
    some `threshold` with a high probability.

    Based on Rajaraman, "Mining of Massive Datasets":
    
    1. Generate items hash signatures
    2. Use LSH to map similar signatures to same buckets
    3. Use UnionFind to merge buckets containing same values
    """
    
    def __init__(self, threshold=0.5, size=10):
        """
        The `size` parameter controls the number of hash
        functions ("signature size") to create.
        """
        self.size = size
        self.unions = UnionFind()
        self.signer = MinHashSignature(size)
        self.hasher = LSH(size, threshold)
        self.hashmaps = [
            defaultdict(list) for _ in range(self.hasher.get_n_bands())
        ]

    def add(self, item, label=None):
        """
        Add an `item` to the cluster.
        
        Optionally, use a `label` to reference this `item`.
        Otherwise, the `item` itself is used as the label.
        """
        # Ensure label for this item
        if label is None:
            label = item

        # Add to unionfind structure
        self.unions[label]

        # Get item signature
        sig = self.signer.sign(item)

        # Unite labels with the same LSH keys in the same band
        for band_idx, hashval in enumerate(self.hasher.hash(sig)):
            self.hashmaps[band_idx][hashval].append(label)
            self.unions.union(label, self.hashmaps[band_idx][hashval][0])

    def groups(self):
        """
        Get the clustering result.
        
        Returns sets of labels.
        """
        return self.unions.sets()

    def match(self, item):
        """
        Get a set of matching labels for `item`.
        
        Returns a (possibly empty) set of labels.
        """
        # Get signature
        sig = self.signer.sign(item)
        
        matches = set()
        
        for band_idx, hashval in enumerate(self.hasher.hash(sig)):
            if hashval in self.hashmaps[band_idx]:
                matches.update(self.hashmaps[band_idx][hashval])
        
        return matches

### An unigram example case

In [82]:
dictionary = Cluster(.75, 30)
dictionary.add('string')
dictionary.match('strang')

set()

Note how the hasher has calculated a bandwith to achieve an (exact) threshold as close as possible to the desired (given) threshold and MinHash signature size.

In [83]:
dictionary.hasher.exact_threshold, dictionary.hasher.bandwidth

(0.765, 6)

In [90]:
jaccard('strang', 'string')

0.714

In [84]:
dictionary.add('strang')
dictionary.match('strang')

{'strang'}

This next example does not match (do you know why?):

In [85]:
dictionary.add('word')
dictionary.match('wird')

set()

Right, the Jaccard similarity is too low and they end in different buckets:

In [86]:
jaccard('wird', 'word')

0.600

Even if you add the word to the dictionary, it will be put into different groups:

In [87]:
dictionary.add('wird')
dictionary.groups()

[['word'], ['string'], ['wird'], ['strang']]

In [91]:
dictionary.match('wird')

{'wird'}

In [92]:
dictionary.add('strangled')

In [93]:
dictionary.match('stringled')

{'strangled', 'string'}

In [94]:
dictionary.groups()

[['word'], ['string'], ['wird'], ['strang'], ['strangled']]

### A bigram case

In [95]:
dictionary = Cluster(.25, 30)
dictionary.add(frozenset(shingle('string', 2)), 'string')
dictionary.match(frozenset(shingle('strang', 2)))

{'string'}

In [97]:
jaccard(frozenset(shingle('string', 2)), frozenset(shingle('strang', 2)))

0.429

In [98]:
dictionary.hasher.exact_threshold, dictionary.hasher.bandwidth

(0.258, 2)

And so forth... You might want to experiment on your own.

### Conclusion

With LSH we can match sets of items by their Jaccard-based set similarity. Note once more that this technique isn't limited to matching similar tokens and is mostly used to identify similar texts, as discussed in the lecture. So instead of character-n-grams, the input might as well be word-n-grams generated from a document.

## Probabilities and underflows

Imagine we have the following three sets of (joint) probabilities; To make this plausible, lets say, they are for the a hundred tokens in our document that our feature selection procedure has elected. And the different probabilities assigned to each token are due to the three different labels we might assign to the document (I.e., we are trying to label a document with one of three different labels, and will assign it the label that has the highest joint probability over all tokens, much like a multinomial Naive Bayes classifier.)

In [99]:
probabilities = [
    [1e-4]*100, [1e-5]*100, [1e-8]*100
]

So as you see immediately, the document should be assigned the last label with the highest probabilities. But lets try calculate actual probabilities for each of the three labels:

In [100]:
pow(1e-4, 100)

0.000

In [101]:
pow(1e-5, 100)

0.000

In [102]:
pow(1e-8, 100)

0.000

Whoops! We quickly reached the limit of our processing capabilites; Two of the three cases give us a wrong answer (`P=0`). This is typically remedied with a simple trick: Work in log-transformed space:

In [103]:
from math import log, exp

In [104]:
A = sum([log(1e-4)]*100); A

-921.034

In [105]:
B = sum([log(1e-5)]*100); B

-1151.293

In [106]:
C = sum([log(1e-8)]*100); C

-1842.068

Note, though, that we cannot move back into normal space:

In [107]:
exp(A)

0.000

So how do we calculate the normalized probability for each of the three labels? I.e., how to get probability scores for the three labels, that, when summed up, equal 1, that is, `P(a) = a / (a+b+c) = exp(A) / (exp(A) + exp(B) + exp(C))` if `A = log(a)` (a.s.f.)? We cannot do that directly, unless we get zeros, after all!

In [108]:
logP = [A, B, C]
#
# Here is the critical trick: subtract the max. log(P_i) value from all values:
#
normP = [i - max(logP) for i in logP]
#
# Result:
#
normP

[0.000, -230.259, -921.034]

This means, the normalized `log(P_i)` for the label with the max. probablity now is zero (i.e., its `P_i = 1`), and all other results are adjusted accordingly to this norm. Now we can safely transform back into our real space; Even if a number now is zero (as is the case for our thrid label), you can in most cases happily ignore that underflow problem, as that probablity is so infinitesimaly small that for all practical concerns it can be considered zero.

In [116]:
expP = list(map(exp, normP)); expP

[1.000000e+00, 1.000000e-100, 0.000000e+00]

In [117]:
P = [i / sum(expP) for i in expP]; P

[1.000000e+00, 1.000000e-100, 0.000000e+00]

Note that there is still a tiny margin of floating-point error in this result, becasue the above result is strictly summed up, does not sum to one, but to about `1 + 10e-101`... But that's not even testable (It might become noticable if the three label probabilities had been a closer call, although - so usually, you will need to check floating point numbers according to some precision you wish to ensure to make sure that your probabilities are all correct):

In [118]:
sum(P) == 1.0

True

But unless you care about being correct beyond 100 digits in the case above, the outcome is probably acceptable in 99.99% of all practical cases (I.e., all but financial institutions will be happy with this "imperfect" solution...)